# 1. Introduction

# 2. Methodology

# 3. Experimental Setup

In [31]:
import pandas as pd

## 3.1. Load Data
* It’s the SMS Spam Collection Data Set from UCI Machine Learning Repository.
* The link of the dataset is as below.
	https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
* The dataset contains one set of SMS messages in English of 5,574 messages, tagged acording being ham (legitimate) or spam. 

In [32]:
data = pd.read_table('data/SMSSpamCollection', sep='\t', names=['label', 'sms_message'])

#print first 5 records
data.head()

,label,sms_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## 3.2. Data Prepocessing
### 3.2.1. Convert Label
The label is converted into binary variable for convenience of calculating
* 0 : "ham"
* 1 : "spam"

In [33]:
# show size of data (#row,#column)
print(data.shape)

# convert label
data['label'] = data.label.map({'ham':0, 'spam':1})

# show label of first 5 records 
data['label'].head()

(5572, 2)


0    0
1    0
2    1
3    0
4    0
Name: label, dtype: int64

### 3.2.2. Split into Traing Set and Testing Set
The method train_test_split of sklearn is used to implement division of traing set and testing set. The result is received by following variables:
* X_train: training data of 'sms_message'
* y_train: training data of 'label'
* X_test: testing data of 'sms_message'
* y_test: testing data of 'label'

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data['sms_message'], 
                                                    data['label'], 
                                                    random_state=1)
print('Number of rows in the total set: {}'.format(df.shape[0]))
print('Number of rows in the training set: {}'.format(X_train.shape[0]))
print('Number of rows in the test set: {}'.format(X_test.shape[0]))

Number of rows in the total set: 5572
Number of rows in the training set: 4179
Number of rows in the test set: 1393


### 3.2.3. Convert SMS Message
SMS message is converted into "Bag of Words"
* Bag of words(BoW)
 * It indicates that the problem to be solved has "a lot of words" or a lot of text data.
 * The basic concept of BoW is to take a piece of text and calculate how often words appear in the text.
 * BoW treats each word equally, and the order in which the words appear is not important.
* The collection of SMS message will be converted into a frequency matrix, each SMS message is a row, each word(token) is a column, and the corresponding (row,column) value is the frequency of each word or token that appears in this document.
* sklearn.feature_extraction.text.CountVectorizer is used to implements BoW
 * It tokenizes the string (dividing the string into a single word) and sets an integer ID for each token.
 * It counts the number of occurrences of each token.
 * Setting parameter 'lowercase' to 'True' can convert token into lowercase, which help ignore case.
 * The default value of parameter 'token_pattern' help ignore punctuation.
 * Setting parameter 'stop_words' to 'english' can ignore the most commonly used words in English, including "am", "an", "and", "the", and so on, which reduce bias of analysis result.

In [42]:
from sklearn.feature_extraction.text import CountVectorizer

count_vector = CountVectorizer()

# Fit the training data and then return the matrix
training_data = count_vector.fit_transform(X_train)

# Transform testing data and return the matrix. Note we are not fitting the testing data into the CountVectorizer()
testing_data = count_vector.transform(X_test)

## 3.3. Algorithms

### 3.3.1. Naive Bayes

## 3.4. Evaluation

# 4. Experimental Results

# 5. Level of Contribution from Each Member